In [7]:
import torch
import torch.nn as nn
import torch.nn.functional
import pandas as pd 
import numpy as np

In [8]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(MLP, self).__init__()

        self.features = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
        )

        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, out_size),
            nn.ReLU(),
        )

    def forward(self, X):

        hidden = self.features(X)
        output = self.classifier(hidden)

        return output

In [9]:
dados = ["se_completo", "se_corolla", "se_onix", "sp_completo", "sp_fit", "sp_hb20", "sp_ka"]
# dados = ["sp_hb20"]

In [10]:
for dado in dados:
  modelo = torch.load(f"./resultados/{dado}/modelo_{dado}")
  modelo.eval()
  dataset = pd.read_csv(f"./dados/{dado}.csv")
  valor = dataset['valor']
  del dataset['valor']
  dif = []
  for index, row in dataset.iterrows():
    tensor = torch.FloatTensor(row.tolist())
    pred = modelo.forward(tensor).item()
    dif.append(valor[index]-pred)
  print(f"{dado} -> {np.array(dif).mean()}")
  print(f"{dado} -> {np.array(dif).max()}")
  print(f"{dado} -> {np.array(dif).min()}")

  print("-------------------------------------------------------------------")
  

se_completo -> 11200.996280515466
se_completo -> 472213.95703125
se_completo -> -67101.609375
-------------------------------------------------------------------
se_corolla -> 2537.4824501811595
se_corolla -> 22690.15625
se_corolla -> -26719.03125
-------------------------------------------------------------------
se_onix -> 7718.177995648301
se_onix -> 38779.6796875
se_onix -> -42984.0078125
-------------------------------------------------------------------
sp_completo -> 14863.277744489065
sp_completo -> 714743.84375
sp_completo -> -149301.109375
-------------------------------------------------------------------
sp_fit -> 13318.205100446428
sp_fit -> 63212.36328125
sp_fit -> -39937.21875
-------------------------------------------------------------------
sp_hb20 -> -48.543192455947135
sp_hb20 -> 9968.11328125
sp_hb20 -> -14443.54296875
-------------------------------------------------------------------
sp_ka -> -159.3755164688082
sp_ka -> 10308.0078125
sp_ka -> -14054.30078125
----

In [11]:
dado = dados[5]
modelo = torch.load(f"./resultados/{dado}/modelo_{dado}")
modelo.eval()
dataset = pd.read_csv(f"./dados/{dado}.csv")
valor = dataset['valor']
del dataset['valor']
dif = []
percent = []

ypred = []
yreal = []

for index, row in dataset.iterrows():
  tensor = torch.FloatTensor(row.tolist())
  pred = modelo.forward(tensor).item()

  ypred.append(pred)
  yreal.append(valor[index])

  dif.append(valor[index]-pred)
  percent.append((pred)*100/valor[index] - 100)

media = np.array(yreal).mean()
mediaPrevista = np.array(ypred).mean()
print(dado)
print(f"Média Real: {media}")
print(f"Média Prevista: {mediaPrevista}")
print()

part1 = 0
part2 = 0
for i in range(len(ypred)):
  part1 += (yreal[i] - ypred[i]) ** 2
  part2 += (yreal[i] - media) ** 2

r2_score = 1 - (part1/part2)
print("R² Score")
print(r2_score*100)
print()

print("Mean Error")
print(np.array(dif).mean())

sp_hb20
Média Real: 44953.337004405286
Média Prevista: 45001.88019686123

R² Score
20.96211665944181

Mean Error
-48.543192455947135
